In [1]:
import miniimagenet_train_few_shot_with_one_hot_vector as ot
import task_generator as tg
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    

In [2]:
FEATURE_DIM = 64  # args.feature_dim
RELATION_DIM = 8  # args.relation_dim
CLASS_NUM = 5  # args.class_num
SAMPLE_NUM_PER_CLASS = 5  # args.sample_num_per_class
BATCH_NUM_PER_CLASS = 15  # args.batch_num_per_class
EPISODE = 1000000  # args.episode
TEST_EPISODE = 1000  # args.test_episode
LEARNING_RATE = 0.001  # args.learning_rate
# GPU = # args.gpu
HIDDEN_UNIT = 10  # args.hidden_unit

# Test just 1 iteration to know how it works

In [3]:
# * Step 1: init data folders
print("init data folders")

# * Init character folders for dataset construction
metatrain_character_folders, metatest_character_folders = tg.mini_imagenet_folders()

init data folders


In [4]:
print("init neural networks")

feature_encoder = ot.CNNEncoder()
relation_network = ot.RelationNetwork(FEATURE_DIM, RELATION_DIM)

feature_encoder.train()
relation_network.train()

feature_encoder.apply(ot.weights_init)
relation_network.apply(ot.weights_init)

feature_encoder.to(device)
relation_network.to(device)
    
feature_encoder_optim = torch.optim.Adam(feature_encoder.parameters(), lr=LEARNING_RATE)
#feature_encoder_scheduler = StepLR(feature_encoder_optim, step_size=100000, gamma=0.5)

relation_network_optim = torch.optim.Adam(relation_network.parameters(), lr=LEARNING_RATE)
#relation_network_scheduler = StepLR(relation_network_optim, step_size=100000, gamma=0.5)

if os.path.exists(str("./models/omniglot_feature_encoder_" + str(CLASS_NUM) + "way_" + str(SAMPLE_NUM_PER_CLASS) + "shot.pkl")):
    feature_encoder.load_state_dict(torch.load(str("./models/miniimagenet_feature_encoder_" + str(CLASS_NUM) + "way_" + str(SAMPLE_NUM_PER_CLASS) + "shot.pkl")))
    print("load feature encoder success")
if os.path.exists(str("./models/omniglot_relation_network_" + str(CLASS_NUM) + "way_" + str(SAMPLE_NUM_PER_CLASS) + "shot.pkl")):
    relation_network.load_state_dict(torch.load(str("./models/miniimagenet_relation_network_" + str(CLASS_NUM) + "way_" + str(SAMPLE_NUM_PER_CLASS) + "shot.pkl")))
    print("load relation network success")

init neural networks


In [20]:
task = tg.MiniImagenetTask(metatrain_character_folders, CLASS_NUM, SAMPLE_NUM_PER_CLASS, 15)
sample_dataloader = tg.get_mini_imagenet_data_loader(task, num_per_class=SAMPLE_NUM_PER_CLASS, split="train", shuffle=False)
batch_dataloader = tg.get_mini_imagenet_data_loader(task, num_per_class=5, split="test", shuffle=False)

# * sample datas
# samples, sample_labels = sample_dataloader.__iter__().next()
# batches, batch_labels = batch_dataloader.__iter__().next()

samples, sample_labels = next(iter(sample_dataloader))
batches, batch_labels = next(iter(batch_dataloader))

samples, sample_labels = samples.to(device), sample_labels.to(device)
batches, batch_labels = batches.to(device), batch_labels.to(device)

In [24]:
batches, batch_labels = next(iter(batch_dataloader))
print(batch_labels)

In [13]:
print(samples.shape)

torch.Size([25, 3, 84, 84])


In [14]:
print(sample_labels.shape)

torch.Size([25])


In [15]:
print(batches.shape)

torch.Size([25, 3, 84, 84])


In [16]:
print(batch_labels.shape)

torch.Size([25])


In [17]:
print(batch_labels)

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4,
        4], device='cuda:0')


In [18]:
batches, batch_labels = next(iter(batch_dataloader))
batch_labels

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4,
        4])